In [25]:
import gradio as gr
import pickle

In [26]:
with open(f"./model/model.pkl", "rb") as f:
    model = pickle.load(f)

In [ ]:
def get_yes_no_question(yes_no):
    yes_no_mapping = {
        "No" : 0,
        "Yes" : 1
    }
    return yes_no_mapping.get(yes_no, -1)

def get_gender(gender):
    gender_mapping = {
        "Female" : 0,
        "Male" : 1
    }
    return gender_mapping.get(gender, -1)

def get_mother_occupation(mother):
    mother_occupation_mapping = {
        "Student" : 0,
        "Specialists in Intellectual and Scientific Activities" : 2,
        "Intermediate Level Technicians and Professions" : 3,
        "Administrative Staff" : 4,
        "Personal Services, Security and Safety Workers and Sellers" : 5,
        "Skilled Workers in Industry, Construction and Craftsmen" : 7,
        "Unskilled Workers" : 9, 
        "Other" : 999
    }
    return mother_occupation_mapping.get(mother, -1)

status = {
    0 : "Dropout",
    1 : "Enrolled",
    2 : "Graduate"
}

def predict_status(_sem_enrolled, _scholarship_holder, _sem_approved, _sem_credited, 
                _tuition_fees, _sem_evaluations, _gender, _admission_grade, 
                _age, _sem_without_evaluations, _mothers_occupation, _gdp):
    scholarship_holder = get_yes_no_question(_scholarship_holder)
    tuition_fees = get_yes_no_question(_tuition_fees)
    gender = get_gender(_gender)
    mothers_occupation = get_mother_occupation(_mothers_occupation)
    data = [[_sem_enrolled, scholarship_holder, _sem_approved, _sem_credited,
             tuition_fees, _sem_evaluations, gender, _admission_grade,
             _age, _sem_without_evaluations, mothers_occupation, _gdp]]
    prediction = model.predict(data)[0]
    prediction_proba = model.predict_proba(data)[0][prediction]
    if prediction == 0:
        return f"The student might {status[prediction]}, model confidence is {prediction_proba:.2f}"
    if prediction == 1:
        return f"The student is still {status[prediction]}, model confidence is {prediction_proba:.2f}"
    if prediction == 2:
        return f"The student should {status[prediction]}, model confidence is {prediction_proba:.2f}"

In [28]:
with gr.Blocks(title="Student Status Prediction") as demo:
    gr.Markdown("""
                # 🎒 Student Status Prediction
                # Dicoding - Solving Educational Institution Problem
                ## Made by : Muhammad Hafizh Dzaki
                ## Gihub Repo : [Here](https://github.com/hfzdzakii/Dicoding-SolvingEducationIntsituteProblem)
                """)
    with gr.Row():
        with gr.Column():
            gr.Markdown("### Input Variables")
            sem_enrolled = gr.Number(label="Sum of 2nd Semester Curricular Units Enrolled:",
                                     value=0, minimum=0, maximum=24)
            sem_approved = gr.Number(label="Sum of 2nd Semester Curricular Units Approved:",
                                     value=0, minimum=0, maximum=24)
            sem_credited = gr.Number(label="Sum of 2nd Semester Curricular Units Credited:",
                                     value=0, minimum=0, maximum=24)
            sem_evaluations = gr.Number(label="Sum of 2nd Semester Curricular Units Evalutions:",
                                        value=0, minimum=0)
            sem_without_evaluations = gr.Number(label="Sum of 2nd Semester Curricular Units Without Evalutions:",
                                                value=0, minimum=0)
            scholarship_holder = gr.Radio(label="Scholarship Holder?",
                                          choices=["No", "Yes"], value="No")
            tuition_fees = gr.Radio(label="Tuition Fees Payed?",
                                    choices=["No", "Yes"], value="No")
            gender = gr.Radio(label="Gender:", 
                              choices=["Male", "Female"], value="Male")
            admission_grade = gr.Number(label="Admission Grade:",
                                        value=0, minimum=0, maximum=200)
            age = gr.Number(label="Age:",
                            value=18, minimum=18)
            mothers_occupation = gr.Dropdown(label="Mother Occupation:",
                                             choices=["Student", "Specialists in Intellectual and Scientific Activities", "Intermediate Level Technicians and Professions",
                                                      "Administrative Staff", "Personal Services, Security and Safety Workers and Sellers",
                                                      "Skilled Workers in Industry, Construction and Craftsmen", "Unskilled Workers", "Other"],
                                             value="Student")
            gdp = gr.Number(label="Gross Domestic Product (GDP):",
                            value=0)
            
        with gr.Column():
            gr.Markdown("""### Example Data
                        Choose one from list below to fill input immediately!
                        """)
            gr.Examples(
                examples=[
                    [6, "No", 5, 0, "No", 13, "Female", 120, 21, 1, "Unskilled Workers", -4.06],
                    [5, "No", 0, 0, "Yes", 0, "Male", 121, 19, 0, "Personal Services, Security and Safety Workers and Sellers", 1.79],
                    [7, "No", 6, 2, "Yes", 10, "Female", 100, 37, 0, "Other", 0.79],
                    [5, "No", 3, 0, "Yes", 9, "Female", 120, 18, 0, 'Administrative Staff', -1.7],
                    [6, "Yes", 6, 0, "Yes", 6, 'Female', 142, 18, 0, "Personal Services, Security and Safety Workers and Sellers", -3.12],
                    [6, "Yes", 6, 2, "Yes", 6, "Female", 137, 23, 0, "Administrative Staff", 1.79]
                ],
                inputs=[sem_enrolled, scholarship_holder, sem_approved,
                         sem_credited, tuition_fees, sem_evaluations, gender,
                         admission_grade, age, sem_without_evaluations, mothers_occupation,
                         gdp]
            )
            
            gr.Markdown("### Predict and Result")
            predict_button = gr.Button("Predict", variant="primary")
            prediction = gr.Textbox(label="Prediction", interactive=False)
            
    predict_button.click(
        fn=predict_status,
        inputs=[sem_enrolled, scholarship_holder,
                sem_approved,
                sem_credited, tuition_fees,
                sem_evaluations, gender,
                admission_grade, age,
                sem_without_evaluations,
                mothers_occupation, gdp],
        outputs=prediction,
    )

demo.launch(debug=True)

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Keyboard interruption in main thread... closing server.
